In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [17]:
data=pd.read_csv(r"E:\Masters\Fall 2021\DA\train.csv")

In [18]:
data

,Employee ID,Date of Joining,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate
0,fffe32003000360033003200,2008-09-30,Female,Service,No,2.0,3.0,3.8,0.16
1,fffe3700360033003500,2008-11-30,Male,Service,Yes,1.0,2.0,5.0,0.36
2,fffe31003300320037003900,2008-03-10,Female,Product,Yes,2.0,NaN,5.8,0.49
3,fffe32003400380032003900,2008-11-03,Male,Service,Yes,1.0,1.0,2.6,0.20
4,fffe31003900340031003600,2008-07-24,Female,Service,No,3.0,7.0,6.9,0.52
...,...,...,...,...,...,...,...,...,...
22745,fffe31003500370039003100,2008-12-30,Female,Service,No,1.0,3.0,NaN,0.41
22746,fffe33003000350031003800,2008-01-19,Female,Product,Yes,3.0,6.0,6.7,0.59
22747,fffe390032003000,2008-11-05,Male,Service,Yes,3.0,7.0,NaN,0.72
22748,fffe33003300320036003900,2008-01-10,Female,Service,No,2.0,5.0,5.9,0.52


In [19]:
df=data.copy()

In [20]:
#droping employee Id column
df= df.drop("Employee ID", axis=1)

In [21]:
#checking the null values 
df.isna().sum()

Date of Joining            0
Gender                     0
Company Type               0
WFH Setup Available        0
Designation                0
Resource Allocation     1381
Mental Fatigue Score    2117
Burn Rate               1124
dtype: int64

In [22]:
#dropping the rows with null values for Resource Allocation, Mental Fatigue Score and Burn Rate
missing_rows=df.loc[df['Resource Allocation'].isna(), :].index
df= df.drop(missing_rows, axis=0).reset_index(drop=True)

missing_rows=df.loc[df['Burn Rate'].isna(), :].index
df= df.drop(missing_rows, axis=0).reset_index(drop=True)

missing_rows=df.loc[df['Mental Fatigue Score'].isna(), :].index
df= df.drop(missing_rows, axis=0).reset_index(drop=True)



In [23]:
df.isna().sum()

Date of Joining         0
Gender                  0
Company Type            0
WFH Setup Available     0
Designation             0
Resource Allocation     0
Mental Fatigue Score    0
Burn Rate               0
dtype: int64

In [24]:
df['Date of Joining']=pd.to_datetime(df['Date of Joining'])
df['Day of Joining']= df['Date of Joining'].apply(lambda x : x.day)
df['Month of Joining']= df['Date of Joining'].apply(lambda x : x.month)
df['Year of Joining']= df['Date of Joining'].apply(lambda x : x.year)

In [25]:
{column : len(df[column].unique()) for column in df.columns }

{'Date of Joining': 366,
 'Gender': 2,
 'Company Type': 2,
 'WFH Setup Available': 2,
 'Designation': 6,
 'Resource Allocation': 10,
 'Mental Fatigue Score': 101,
 'Burn Rate': 101,
 'Day of Joining': 31,
 'Month of Joining': 12,
 'Year of Joining': 1}

In [26]:
len(df['Year of Joining'].unique())

1

In [27]:
df= df.drop('Year of Joining', axis=1)
df= df.drop('Date of Joining', axis=1)

In [28]:
# Binary Encoding 

df['Gender']= df['Gender'].replace({'Female':0, 'Male': 1})
df['Company Type']= df['Company Type'].replace({'Product':0, 'Service': 1})
df['WFH Setup Available']= df['WFH Setup Available'].replace({'No':0, 'Yes': 1})

In [29]:
df

,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate,Day of Joining,Month of Joining
0,0,1,0,2.0,3.0,3.8,0.16,30,9
1,1,1,1,1.0,2.0,5.0,0.36,30,11
2,1,1,1,1.0,1.0,2.6,0.20,3,11
3,0,1,0,3.0,7.0,6.9,0.52,24,7
4,1,0,1,2.0,4.0,3.6,0.29,26,11
...,...,...,...,...,...,...,...,...,...
18585,0,0,1,1.0,3.0,6.0,0.48,15,12
18586,1,0,0,3.0,7.0,6.2,0.54,27,5
18587,0,0,1,3.0,6.0,6.7,0.59,19,1
18588,0,1,0,2.0,5.0,5.9,0.52,10,1


In [350]:
# Split the data into x and y

y= df['Burn Rate']
x= df.drop('Burn Rate', axis=1)


#Train Test Split
x_train, x_test, y_train, y_test= train_test_split(x,y,train_size=0.8,random_state=1)

In [351]:
scaler=StandardScaler()
scaler.fit(x_train)
x_train=pd.DataFrame(scaler.transform(x_train), index= x_train.index, columns=x_train.columns)
x_test=pd.DataFrame(scaler.transform(x_test), index= x_test.index, columns=x_test.columns)

In [352]:
#Training Linear Regression

In [353]:
LR= LinearRegression().fit(x_train, y_train)
y_train_pred_LR= LR.predict(x_train)

In [354]:

print("The coeffients for linear regression are " , LR.coef_)
print("The intercept for Linear Regression is : " , LR.intercept_)


The coeffients for linear regression are  [ 0.00086514  0.00086514  0.00086514 -0.0057159   0.0597891   0.14292723
 -0.00056358 -0.00024144]
The intercept for Linear Regression is :  0.4523043302850995


In [355]:
# Training error


print("R^2 for Training Set : " , LR.score(x_train,y_train))
print("Mean Absolute Error for Training Set :", mean_absolute_error(y_train, y_train_pred_LR))
print("Mean square Error for Training Set:", mean_squared_error(y_train,y_train_pred_LR))
print("Root Mean square Error for Training Set:", sqrt(mean_squared_error(y_train,y_train_pred_LR)))

R^2 for Training Set :  0.9206073223345814
Mean Absolute Error for Training Set : 0.04563531949896247
Mean square Error for Training Set: 0.0031167654670345743
Root Mean square Error for Training Set: 0.05582799895244835


In [356]:
#Predicting the values for test set
y_pred_LR=LR.predict(x_test)

In [357]:
#Test Errors for Linear Regression 

MAE_LR= mean_absolute_error(y_test, y_pred_LR)
MSE_LR=mean_squared_error(y_test,y_pred_LR)
RMSE_LR= sqrt(mean_squared_error(y_test,y_pred_LR))
R2=round(LR.score(x_test,y_test),2)

print("R^2 for Test Set : " , R2)
print("Mean Absolute Error for Test Set :", MAE_LR)
print("Mean square Error for Test Set:", MSE_LR)
print("Root Mean square Error for Test Set:", RMSE_LR)


R^2 for Test Set :  0.92
Mean Absolute Error for Test Set : 0.04640970525020264
Mean square Error for Test Set: 0.003212345462220203
Root Mean square Error for Test Set: 0.05667755695352617


In [358]:
#Training Random Forest Regression 

RF= RandomForestRegressor().fit(x_train, y_train)
y_train_pred_RF=RF.predict(x_train)
y_pred_RF=RF.predict(x_test)


In [359]:
# Training error

print("Mean Absolute Error for Training Set :", mean_absolute_error(y_train, y_train_pred_RF))
print("Mean square Error for Training Set:", mean_squared_error(y_train,y_train_pred_RF))
print("Root Mean square Error for Training Set:", sqrt(mean_squared_error(y_train,y_train_pred_RF)))

Mean Absolute Error for Training Set : 0.01750982667973054
Mean square Error for Training Set: 0.000501686893153037
Root Mean square Error for Training Set: 0.022398368091292656


In [360]:
#Test Error
MAE_RF= mean_absolute_error(y_test, y_pred_RF)
MSE_RF=mean_squared_error(y_test,y_pred_RF)
RMSE_RF= sqrt(mean_squared_error(y_test,y_pred_RF))

print("Mean Absolute Error for Test Set :", MAE_RF)
print("Mean square Error for Test Set:", MSE_RF)
print("Root Mean square Error for Test Set:", RMSE_RF)

Mean Absolute Error for Test Set : 0.0455795258694297
Mean square Error for Test Set: 0.003267373318162707
Root Mean square Error for Test Set: 0.05716094224348219


In [361]:
#Training SVR
SV= SVR().fit(x_train, y_train)
y_train_pred_SVR= SV.predict(x_train)
y_pred_SVR=SV.predict(x_test)

In [362]:
# Training error

print("Mean Absolute Error for Training Set :", mean_absolute_error(y_train, y_train_pred_SVR))
print("Mean square Error for Training Set:", mean_squared_error(y_train,y_train_pred_SVR))
print("Root Mean square Error for Training Set:", sqrt(mean_squared_error(y_train,y_train_pred_SVR)))

Mean Absolute Error for Training Set : 0.04916190361638224
Mean square Error for Training Set: 0.003336128762526102
Root Mean square Error for Training Set: 0.057759230972426405


In [363]:
# Test error for SVR

MAE_SVR= mean_absolute_error(y_test, y_pred_SVR)
MSE_SVR=mean_squared_error(y_test,y_pred_SVR)
RMSE_SVR= sqrt(mean_squared_error(y_test,y_pred_SVR))

print("Mean Absolute Error for Test Set :", MAE_SVR)
print("Mean square Error for Test Set:", MSE_SVR)
print("Root Mean square Error for Test Set:", RMSE_SVR)

Mean Absolute Error for Test Set : 0.04944618805577363
Mean square Error for Test Set: 0.0034056033686410726
Root Mean square Error for Test Set: 0.05835754765787432


In [364]:


Errors ={  "Mean absolute Error":  [MAE_LR,MAE_SVR,MAE_RF], 
            "Mean Square Error":   [MSE_LR,MSE_SVR,MSE_RF],
            "Root Mean Square Error": [RMSE_LR,RMSE_SVR,RMSE_RF]
         }



In [365]:
#comparison of errors for different models

for key,value in Errors.items():
    print(key, "for Linear Regression =", value[0])
    print(key, "for Support Vector Regression =", value[1])
    print(key, "for Random Forest Regression =", value[2])
    print("\n")
   
    
    

Mean absolute Error for Linear Regression = 0.04640970525020264
Mean absolute Error for Support Vector Regression = 0.04944618805577363
Mean absolute Error for Random Forest Regression = 0.0455795258694297


Mean Square Error for Linear Regression = 0.003212345462220203
Mean Square Error for Support Vector Regression = 0.0034056033686410726
Mean Square Error for Random Forest Regression = 0.003267373318162707


Root Mean Square Error for Linear Regression = 0.05667755695352617
Root Mean Square Error for Support Vector Regression = 0.05835754765787432
Root Mean Square Error for Random Forest Regression = 0.05716094224348219


